In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [2]:
goolam_d=pd.read_csv('count_goolam.csv')
goolam_d1=pd.read_csv('count_goolam.csv')
dpg_d = pd.read_csv('dp_goolam.csv')
simg_d = pd.read_csv('sim_goolam.csv')
goolam=goolam_d.values
goolam_imputed=goolam_d1.values
dpg=dpg_d.values
simg=simg_d.values

In [3]:
goolam

array([[7.26050379, 6.94073311, 7.21484089, ..., 6.50122418, 5.59445082,
        6.16173529],
       [9.38275951, 9.72226659, 9.16331161, ..., 8.28204683, 9.19829928,
        8.48738682],
       [5.76941702, 5.69762434, 4.76441438, ..., 3.92625927, 0.        ,
        3.43963744],
       ...,
       [0.        , 0.        , 0.        , ..., 4.87801294, 4.89126383,
        4.67949524],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        5.0803887 ],
       [4.47469506, 5.06134559, 3.81654264, ..., 9.29235153, 9.2637002 ,
        9.07100873]])

In [4]:
dpg

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.70460902,
        0.        ],
       ...,
       [0.87685918, 0.88466278, 0.88237425, ..., 0.        , 0.        ,
        0.        ],
       [0.88508505, 0.89844864, 0.89460304, ..., 0.9110198 , 0.91613266,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [5]:
simg

array([[0.        , 0.99774619, 0.99606351, ..., 0.97664929, 0.97811409,
        0.97611906],
       [0.99774619, 0.        , 0.99458135, ..., 0.97434835, 0.97699408,
        0.97398718],
       [0.99606351, 0.99458135, 0.        , ..., 0.977172  , 0.9791371 ,
        0.97642788],
       ...,
       [0.97664929, 0.97434835, 0.977172  , ..., 0.        , 0.98488937,
        0.9820425 ],
       [0.97811409, 0.97699408, 0.9791371 , ..., 0.98488937, 0.        ,
        0.98471091],
       [0.97611906, 0.97398718, 0.97642788, ..., 0.9820425 , 0.98471091,
        0.        ]])

In [6]:
dpg[3][0]

0.953282944047711

In [7]:
simg[0][3]

0.994729657589186

In [8]:
nrow = len(goolam)
ncol=len(goolam[0])
nr=len(simg)
print(nrow)
print(ncol)
print(nr)

10057
124
124


In [9]:
imp=np.empty((nrow,ncol))
for i, j in itertools.product(range(0,nrow), range(0,ncol)):
    imp[i][j]=0

In [10]:
imp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
impute=[]
impute=list()
for i, j in itertools.product(range(0,nrow), range(0,ncol)):
    if dpg[i][j]>0.5:
        zval=goolam[i][j]
        temp=[]
        temp=list()
        for k in range(nr):
            if simg[j][k]>0.75:
                val=goolam[i][k]
                temp.append([i,k,val])
        ##regression
        t=np.array(temp)
        n=len(t)
        if(n==0):
            imp[i][j]=np.nan
        if(1<=n<5):
            X=t[:,1]
            y=t[:,2]
            m=round(np.average(y))
            impute.append([i,j,zval,m])
            imp[i][j]=m
            goolam_imputed[i][j]=m
        if(n>=5):
            X=t[:,1]
            y=t[:,2]
            ##Regression
            
            from sklearn.model_selection import train_test_split
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
            X_test=np.append(X_test,j)
            y_test=np.append(y_test,zval)
            
            # Fitting Decision Tree Regression to the dataset
            from sklearn.tree import DecisionTreeRegressor
            regressor = DecisionTreeRegressor(random_state=0)
            regressor.fit(X_train.reshape(-1,1), y_train.reshape(-1,1))
            y_pred = regressor.predict(X_test.reshape(-1,1))
            
            df = pd.DataFrame({'Cell_No':X_test.reshape(-1),'Real_Values':y_test.reshape(-1), 'Imputed_Values':y_pred.reshape(-1)})
            m=df.loc[df['Cell_No'] == j, 'Imputed_Values'].squeeze()
            impute.append([i,j,zval,m])
            imp[i][j]=m
            goolam_imputed[i][j]=m
    else:
        imp[i][j]=np.nan

In [12]:
impute

[[2, 6, 0.0, 5.39765033672992],
 [2, 12, 0.0, 3.78179975510593],
 [2, 13, 0.0, 1.18808020072684],
 [2, 22, 0.0, 2.47394277597351],
 [2, 46, 0.0, 3.83304580843968],
 [2, 48, 0.0, 4.23283600352247],
 [2, 58, 0.0, 4.87449195336856],
 [2, 62, 0.0, 0.0],
 [2, 63, 0.0, 0.0],
 [2, 64, 0.0, 0.0],
 [2, 65, 0.0, 0.0],
 [2, 66, 0.0, 0.0],
 [2, 67, 0.0, 0.0],
 [2, 68, 0.0, 0.0],
 [2, 70, 0.0, 2.15368860593194],
 [2, 71, 0.0, 0.0],
 [2, 72, 0.0, 0.0],
 [2, 73, 0.0, 0.0],
 [2, 74, 0.0, 0.0],
 [2, 78, 0.0, 3.86671456287603],
 [2, 79, 0.0, 0.0],
 [2, 81, 0.0, 1.70295089882925],
 [2, 82, 0.0, 0.0],
 [2, 83, 0.0, 0.0],
 [2, 84, 0.0, 0.0],
 [2, 85, 0.0, 0.0],
 [2, 86, 0.0, 0.0],
 [2, 87, 0.0, 0.0],
 [2, 88, 0.0, 0.0],
 [2, 89, 0.0, 0.0],
 [2, 100, 0.0, 5.37875495305051],
 [2, 102, 0.0, 4.31254512318454],
 [2, 110, 0.0, 1.92194465948288],
 [2, 111, 0.0, 4.08686183393622],
 [2, 113, 0.0, 4.08686183393622],
 [2, 117, 0.0, 4.4717147896464],
 [2, 119, 0.0, 3.31464194833183],
 [2, 122, 0.0, 3.92625927424644],


In [13]:
imp

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan, 3.92625927,
               nan],
       ...,
       [0.        , 0.        , 0.        , ...,        nan,        nan,
               nan],
       [0.        , 0.        , 0.        , ..., 3.31755257, 0.        ,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])

In [14]:
goolam_imputed

array([[7.26050379, 6.94073311, 7.21484089, ..., 6.50122418, 5.59445082,
        6.16173529],
       [9.38275951, 9.72226659, 9.16331161, ..., 8.28204683, 9.19829928,
        8.48738682],
       [5.76941702, 5.69762434, 4.76441438, ..., 3.92625927, 3.92625927,
        3.43963744],
       ...,
       [0.        , 0.        , 0.        , ..., 4.87801294, 4.89126383,
        4.67949524],
       [0.        , 0.        , 0.        , ..., 3.31755257, 0.        ,
        5.0803887 ],
       [4.47469506, 5.06134559, 3.81654264, ..., 9.29235153, 9.2637002 ,
        9.07100873]])

In [15]:
goolam_imputed[2][46]

3.83304580843968

In [16]:
goolam[2][46]

0.0

In [17]:
goolam_imputed_t=np.transpose(goolam_imputed)

In [18]:
goolam_imputed_t.shape

(124, 10057)

In [19]:
import csv

with open("goolam_imputed.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(goolam_imputed)

In [20]:
import csv

with open("goolam_imputed_t.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(goolam_imputed_t)